In [6]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, selectSameLabel, KG_MLAug, test_aug
from processTools import processRealData
import torch
from ModelClassify_2 import ModelClassify
from targeTools import testThresholdFive, Accuracy

In [7]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_bhd.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [8]:
feature_pd

,0,1,2,3,4,5,6,7,8,9,...,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.036818,0.026898,0.010529,-0.087857,0.008474,0.021624,0.037671,-0.042609,-0.033987,0.001863
1,0.0000,0.0025,0.0000,0.0000,0.0025,0.0025,0.0025,0.0025,0.0000,0.0025,...,0.050693,0.058083,0.040556,-0.099527,-0.021974,0.034129,-0.015654,-0.016042,0.018142,0.009868
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,-0.017891,0.025206,0.068204,-0.120173,-0.041826,-0.001013,-0.015208,0.003502,0.038556,0.032665
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,-0.005363,0.065209,0.039654,-0.100884,0.034206,0.002743,-0.040962,0.035129,0.013996,0.079778
4,0.0100,0.0075,0.0075,0.0025,0.0000,0.0000,0.0075,0.0000,0.0100,0.0025,...,-0.019232,-0.013705,0.001783,-0.069890,0.016761,0.059995,-0.020010,-0.052366,-0.008041,-0.017827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,0.0025,0.0075,0.0075,0.0000,0.0075,0.0075,0.0075,0.0075,0.0075,0.0000,...,-0.082709,-0.038805,0.049082,-0.072492,0.061679,0.055330,-0.054334,0.022155,0.081683,0.008963
574,0.0000,0.0025,0.0025,0.0075,0.0100,0.0100,0.0000,0.0025,0.0000,0.0075,...,0.005489,-0.021214,-0.028315,-0.027093,0.051935,0.024115,-0.000847,0.049707,-0.002306,0.000640
575,0.0000,0.0025,0.0025,0.0000,0.0025,0.0025,0.0000,0.0000,0.0000,0.0025,...,-0.021107,0.007253,0.054399,-0.098169,0.005296,0.016531,0.015466,-0.041228,0.019770,-0.002437
576,0.0025,0.0025,0.0125,0.0025,0.0125,0.0025,0.0125,0.0025,0.0000,0.0000,...,-0.052751,-0.001265,0.058449,-0.104947,0.083710,0.017370,-0.021055,0.059610,0.035077,0.055231


In [13]:
smote_multiple = {}
for item in range(1, 10, 2):
    # 处理数据
    multi_label_samples = labels_pd[(labels_pd.sum(axis=1) >= 2)]
    multi_label_indices = multi_label_samples.index
    multi_features_samples = feature_pd.loc[multi_label_indices]
    multi_features_samples = multi_features_samples.drop([43, 522], axis=0)
    multi_label_samples = multi_label_samples.drop([43, 522], axis=0)
    multi_label_samples = multi_label_samples.drop(columns='plastoglobule')
    new_X, new_y = KG_MLAug(multi_features_samples, multi_label_samples, item)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/DPC_T5_test_mutil_label_122_finally.csv")

    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 1424)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1424)

    # 合并生成的多标签
    G_feature = pd.concat([G_feature, new_X], axis=0)
    G_label = pd.concat([G_label, new_y], axis=0)

    new_testX, new_testY = test_aug(test_df, item)
    G_feature = pd.concat([G_feature, new_testX], axis=0)
    G_label = pd.concat([G_label, new_testY], axis=0)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)
    print(train_feature.shape)



    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1424].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify()
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(200):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
        print(f"Epoch [{epoch+1}/{200}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

(1263, 1424)
Epoch [1/200], Average Loss: 0.7203, ACC: 0.24907627342306676
epoch:0, bestThreshold:[0.48, 0.49, 0.5, 0.01], GM:0.4050273895263672, OAA:0.0, ACC:0.29986338797814205, F1:0.45273253321647644
Epoch [2/200], Average Loss: 0.7157, ACC: 0.2631301134864077
epoch:1, bestThreshold:[0.01, 0.49, 0.51, 0.01], GM:0.4069399833679199, OAA:0.01639344262295082, ACC:0.30669398907103806, F1:0.454098641872406
Epoch [3/200], Average Loss: 0.7102, ACC: 0.2769200316706251
epoch:2, bestThreshold:[0.47, 0.51, 0.5, 0.01], GM:0.431748628616333, OAA:0.0, ACC:0.3415300546448087, F1:0.5002732872962952
Epoch [4/200], Average Loss: 0.6998, ACC: 0.2794272895223008
epoch:3, bestThreshold:[0.01, 0.5, 0.01, 0.01], GM:0.40754103660583496, OAA:0.0, ACC:0.29713114754098363, F1:0.45163968205451965
Epoch [5/200], Average Loss: 0.6910, ACC: 0.3046978094484027
epoch:4, bestThreshold:[0.46, 0.49, 0.01, 0.01], GM:0.4002186357975006, OAA:0.0, ACC:0.2827868852459016, F1:0.4355194866657257
Epoch [6/200], Average Loss: 

In [14]:
smote_multiple

{1: [tensor(0.8383),
  0.8032786885245902,
  0.8360655737704918,
  tensor(0.8484),
  tensor(0.8566),
  tensor(0.8470)],
 3: [tensor(0.8464),
  0.8114754098360656,
  0.8442622950819673,
  tensor(0.8525),
  tensor(0.8689),
  tensor(0.8552)],
 5: [tensor(0.8415),
  0.819672131147541,
  0.8401639344262295,
  tensor(0.8443),
  tensor(0.8566),
  tensor(0.8470)],
 7: [tensor(0.8262),
  0.7868852459016393,
  0.8237704918032787,
  tensor(0.8320),
  tensor(0.8525),
  tensor(0.8361)],
 9: [tensor(0.8202),
  0.7459016393442623,
  0.8155737704918032,
  tensor(0.8238),
  tensor(0.8770),
  tensor(0.8388)]}